Load the data.

In [1]:
# Code adapted from Harish Satgunarajah
import pandas as pd

df = pd.read_csv("../FIBAData.csv", index_col = 0, encoding='latin-1')
df.head()

,Region,Date,Away Team,Home Team,Result,Team,Period,Clock,Player,Play Details,Away Score,Home Score
ID,,,,,,,,,,,,
1,2019 World Cup,9/15/2019,Spain,Argentina,Miss 2 Pts,Spain,1,9:39,Ricky Rubio,9 Ricky Rubio > Hand Off > From Stationary > R...,0,0
2,2019 World Cup,9/15/2019,Spain,Argentina,NaN,NaN,1,9:39,NaN,Shot > Ricky Rubio > Any Type > 2 Point Attemp...,0,0
3,2019 World Cup,9/15/2019,Spain,Argentina,NaN,NaN,1,9:39,NaN,Misc Stat > Defensive Rebound > Marcos Delia,0,0
4,2019 World Cup,9/15/2019,Spain,Argentina,Miss 3 Pts,Argentina,1,9:30,Luis Scola,7 Facundo Campazzo > P&R Ball Handler > High P...,0,0
5,2019 World Cup,9/15/2019,Spain,Argentina,NaN,NaN,1,9:30,NaN,Shot > Luis Scola > Any Type > 3 Point Attempt...,0,0


Let's add a column with an ID for each unique game.

In [2]:
df['GameID'] = df['Region'] + str(df['Date']) + df['Home Team'] + df['Away Team']
games = df['GameID'].unique().tolist()
df['GameID'] = df['GameID'].apply(lambda x: games.index(x))

We can see which teams we have the most/least games for.

In [3]:
HTs = df.groupby(['Home Team'])['GameID'].nunique()
ATs = df.groupby(['Away Team'])['GameID'].nunique()
HTs.add(ATs, fill_value=0).astype(int).sort_values(ascending=False)

Serbia          65
France          64
Spain           63
Lithuania       58
Argentina       53
                ..
Saudi Arabia     3
South Africa     3
Kuwait           2
Singapore        2
Bahrain          1
Name: GameID, Length: 93, dtype: int64

In [4]:
teams = set(df['Home Team'].unique()).union(set(df['Away Team'].unique()))
len(teams)

93

Let's see how many players we have data on

In [5]:
df['Player'].value_counts()

Luis Scola           1286
Bojan Bogdanovic     1151
Bogdan Bogdanovic    1023
Facundo Campazzo      905
Andray Blatche        830
                     ... 
Min Joe Foong           1
Nick Johnson            1
Guilherme Bento         1
Satkeyev Aimurat        1
Mario Lopez             1
Name: Player, Length: 2873, dtype: int64

We seem to have 2873, but I wonder if there are any duplicates in here with slight differences in spelling (e.g. capitalization, accents, middle names). However, cleaning this seems difficult...

In [6]:
import unidecode
player_names = set(df['Player'].unique())
decoded_player_names = set(df['Player'].apply(lambda x: unidecode.unidecode(str(x))).unique())
capital_player_names = set(df['Player'].str.upper().unique())
capital_decoded_player_names = set(df['Player'].apply(lambda x: unidecode.unidecode(str(x).upper())).unique())
print(len(player_names), len(decoded_player_names))
print(len(capital_player_names), len(capital_decoded_player_names))
print(player_names - decoded_player_names)
print(decoded_player_names - player_names)

2874 2870
2860 2856
{nan, 'Sergio Rodríguez Gómez', 'João Paulo Batista', 'Martin Kríz', 'Ondrej BalvÃ\xadn', 'Leonardo Gutiérrez', 'Tomás Satoransky', 'Jakub Kudlácek', 'José-Manuel Calderon', 'Hidayet Türkoglu', 'Andrés Nocioni', 'Sergio Rodríguez', 'Tomás Vyoral', 'José Calderon', 'Fidel Mendoça', 'Martin KrÃ\xadz', 'Ondrej Balvín'}
{'Martin KrAz', 'Jose Calderon', 'Ondrej BalvAn', 'Jose-Manuel Calderon', 'Jakub Kudlacek', 'Leonardo Gutierrez', 'Fidel Mendoca', 'Sergio Rodriguez Gomez', 'Andres Nocioni', 'Sergio Rodriguez', 'Martin Kriz', 'Hidayet Turkoglu', 'nan'}


In [7]:
import difflib #https://docs.python.org/3/library/difflib.html
players = df[df['Team'] == 'Spain']['Player'].unique().tolist()
difflib.get_close_matches('Jose Calderon', players)

['José Calderon', 'José-Manuel Calderon']

Let's see whether we need to clean the period data.

In [8]:
period_list = df.groupby('GameID')['Period'].unique()
print("Periods in data  :", df['Period'].unique().tolist())
print("Number of Games  :", len(period_list))
for i in range(1, 5):
    print("Missing period", i, ":", period_list.apply(lambda x: i not in x).sum())
for (i, j) in ((5, 6), (5, 7), (6, 7)):
    print("Has", j, "but not", i, " :", period_list.apply(lambda x: j in x and i not in x).sum())

Periods in data  : [1, 2, 3, 4, 5, 6, 7]
Number of Games  : 1164
Missing period 1 : 10
Missing period 2 : 33
Missing period 3 : 20
Missing period 4 : 25
Has 6 but not 5  : 12
Has 7 but not 5  : 0
Has 7 but not 6  : 6


We may wish to disregard the games with these exceptions as being unreliable, or we can ignore these errors. Let us also see how many plays we have per period.

In [9]:
df.groupby(['GameID', 'Period']).size().describe()

count    4684.000000
mean      168.519641
std        37.370304
min         6.000000
25%       149.000000
50%       166.000000
75%       183.000000
max       702.000000
dtype: float64

As we see, some periods include only 6 rows of information, while some include up to 702. This is further evidence that the period labels may not be reliable in some cases. In the future, we could analyze the clock column to ensure validity of the period labels.

Below are some ideas for parsing the plays, if we were to do this from scratch.

In [10]:
success_type = {"Make 2 Pts" : 2, "Make 3 Pts" : 3, "1 Pts" : 1, "Made" : 1, "Foul": 0.1}

def play_distribution(player, gameID):
    """
    Return player's play type distribution for given game.
    Synergy actions are PnR Handler, Spot Up, Transition, Isolation, Post Up, PnR Man, Cut, ORB, Off Scree, Hand Off
    player: String, player's full name
    gameID: int, game ID from df
    return: dict, {play_type: [attempts, pts]}
    """
    # TODO Handle non-plays? (free throws, turnovers, fouls, etc.)
    # What result for foul? Count number of shots received? Right now adding 0.1...
    # If assisted by different player, count under resulting play type
    play_distribution = {}
    temp = df[df['GameID'] == gameID]
    temp = temp[temp['Play Details'].str.contains(player)]['Play Details']
    for play in temp.tolist():
        play = play.split(' > ')
        if player in play[0] or 'Ball Delivered' in play:
            play_type = play[1 if player in play[0] else play.index('Ball Delivered') + 2]
            play_result = play[-1]
            if play_type in play_distribution:
                play_distribution[play_type][0] += 1
                play_distribution[play_type][1] += success_type[play_result] if play_result in success_type else 0
            else:
                play_distribution[play_type] = [1, success_type[play_result] if play_result in success_type else 0]
    return play_distribution

play_distribution("Ricky Rubio", 0)

{'Hand Off': [3, 2],
 'P&R Ball Handler': [11, 4.199999999999999],
 'Non Possession': [8, 0],
 'Offensive Rebound': [2, 2],
 'Spot-Up': [3, 5],
 'Cut': [1, 1],
 'No Play Type': [3, 0.1],
 'Transition': [1, 0]}

In [11]:
def play_counts(player, gameID):
    """
    Return player's play type counts for game with given gameID.
    Synergy actions: PnR Handler, Spot Up, Transition, Isolation, Post Up, PnR Man, Cut, ORB, Off Screen, Hand Off
    player: String, player's full name as appears in data
    gameID: int, game ID from df
    return: dict, {play_type: count}
    """
    play_distribution = {}
    temp = df[df['GameID'] == gameID]
    temp = temp[temp['Play Details'].str.contains(player)]['Play Details']
    for play in temp.tolist():
        play = play.split(' > ')
        if player in play[0] or 'Ball Delivered' in play: # If assisted by different player, count under resulting play type
            play_type = play[1 if player in play[0] else play.index('Ball Delivered') + 2]
            if play_type in play_distribution:
                play_distribution[play_type] += 1
            elif play_type != 'Non Possession' and play_type != 'No Play Type':
                play_distribution[play_type] = 1
    return play_distribution

play_counts("Marc Gasol", 0)

{'Transition': 2,
 'P&R Roll Man': 2,
 'Post-Up': 5,
 'Offensive Rebound': 2,
 'Spot-Up': 2,
 'Cut': 2}

In [12]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if len(play) >= 2 and play[1] == 'Non Possession':
        unique.add(play[2])
unique

{'8 Sec Violation',
 'Free Throw',
 'Kicked Ball',
 'Non Shooting Foul',
 'Out of Bound 5 Sec Violation',
 'Shot Clock Violation'}

In [13]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if play[0] == 'Misc Stat':
        unique.add(play[1])
unique

{'Assist',
 'Block',
 'Defensive Rebound',
 'Offensive Rebound',
 'Steal',
 'Team Defensive Rebound',
 'Team Offensive Rebound'}

In [14]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if play[0] == 'Shot':
        unique.add(play[-1])
unique

{'Made', 'Make 2 Pts', 'Make 3 Pts', 'Miss 2 Pts', 'Miss 3 Pts', 'Missed'}

In [15]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if play[0].isalpha():
        unique.add(play[0])
unique

{'Shot', 'Turnover'}

In [16]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if not any(map(str.isdigit, play[0])):
        unique.add(play[0])
unique

{'Misc Stat',
 'Personal Foul',
 'Player Movement',
 'Shot',
 'Technical Foul',
 'Time Event',
 'Turnover'}

In [17]:
player_names= set(df['Player'].dropna().unique())
player_names

{'Iverson Molinar',
 'Omar Farag',
 'Ka-Kui Chow',
 'Iftach Ziv',
 'Bar Timor',
 'Facundo Campazzo',
 'Fadi Ibrahim',
 'Ahmed Soliman',
 'Tae-Young Moon',
 'Duda Sanadze',
 'Carl Wheatle',
 'Oleksandr Ryabchuk',
 'Alexander Renfroe',
 'Omar Mouhli',
 'Matt Janning',
 'Omar Maatouk',
 'Qi Zhou',
 'Xavier Richards',
 'Gen Li',
 'Hjalmar Stefansson',
 'Sergiy Gladyr',
 'Antero Lehto',
 'Gabriel Deck',
 'James Okello',
 'David Barlow',
 'Lingxu Zeng',
 'Amine Rzig',
 'Andrew Albicy',
 'Paul Biligha',
 'Rain Veideman',
 'Suad Sehovic',
 'Carlos Javier Rodriguez',
 'Sergey Karasev',
 'Ricardo Fischer',
 'Nikolay Bazhin',
 'Junfei Ren',
 'Tudor Gheorghe',
 'Grandy Glaze',
 'Wen-Cheng Tsai',
 'Marko Todorovic',
 'Loukas Mavrokefalidis',
 'Aleksa Radanov',
 'Henri Drell',
 'Alexander Kudrautsau',
 'Ante Masic',
 'Nathan Sobey',
 'Pitchou Kambuy',
 'John Jenkins',
 'Ido Kozikaro',
 'Zakaria El Masbahi',
 'Robinson Odoch',
 'Kevin Alas',
 'Juancho Hernangomez',
 'Nikos Gikas',
 'Hugo Martins',
 '

In [18]:
player_counts = {}
for p in player_names:
    if p not in player_counts:
        player_counts[p] = {'P&R Ball Handler':0, 'Spot-Up':0, 'Transition':0, 'ISO':0, 'Post-Up':0, 'P&R Roll Man':0, 'Cut':0, 'Offensive Rebound':0, 'Off Screen':0, 'Hand Off':0, 'Total Counts':0}


In [19]:
#for each player track the counts for each different play type
for p in player_names:
    is_player = df['Player'] == p
    player = df[is_player]
    games = set(player['GameID'].unique())
    for g in games:
        cnts = play_counts(p, g)
        for each in cnts:
            player_counts[p][each] += cnts[each]
    s = 0
    for each in player_counts[p]:
        s += player_counts[p][each]
    player_counts[p]['Total Counts'] = s


In [28]:
player_counts['Kevin Durant']

{'P&R Ball Handler': 36,
 'Spot-Up': 24,
 'Transition': 27,
 'ISO': 19,
 'Post-Up': 7,
 'P&R Roll Man': 0,
 'Cut': 2,
 'Offensive Rebound': 3,
 'Off Screen': 14,
 'Hand Off': 5,
 'Total Counts': 137}